# 8. Resolve data issues

The analysis in section 8.2 is based on data retrieved from LexisNexis (LexisNexis Dossier).

In [1]:
sample_file = '../preprocessed_data/sample_2019-08-24.feather'
issues_to_address = '../preprocessed_data/issues_to_address_2019-08-23.pickle'
pipelines_2010_raw_file = '../data/pipelines_2010_2019-08-16.feather'

## Setup

In [2]:
import pandas as pd
import numpy as np
from datetime import date
from functools import partial

today = date.today().isoformat()

In [3]:
import wrds

db = wrds.Connection(wrds_username='juujian')

Loading library list...
Done


## Load data

In [4]:
import pickle

with open(issues_to_address, 'rb') as file:
    issues = pickle.load(file)
    
issues = (issue for issue in issues)

In [5]:
sample = pd.read_feather(sample_file)
sample.sample(5)

,OPERATOR_ID,YEAR,NAME,COMMODITY,MILES,AGE_UNKNOWN_MILES,MILES_PRE_1940,MILES_1940,MILES_1950,MILES_1960,MILES_1970,MILES_1980,MILES_1990,MILES_2000,MILES_2010,PERC_OFFSHORE,AVG_AGE,PARENT,INCIDENTS,SIGNIFICANT_INCIDENTS
127,4906,2013,EXXONMOBIL PIPELINE CO,non-hvl,493.30,0.0,0.0,3.1,13.090,218.090,13.440,40.160,27.220,47.270,48.020,0.0,40.737713,Exxon Mobil,3.0,0.0
583,30826,2012,WILLIAMS FIELD SERVICES,hvl,375.90,22.0,0.0,0.0,0.000,0.000,0.000,25.000,0.000,1047.000,36.000,0.0,15.126354,The Williams Companies,0.0,0.0
530,26149,2010,ALYESKA PIPELINE SERVICE CO,crude,353.00,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,NaN,Alyeska Pipeline Service,2.0,1.0
593,30829,2012,ENTERPRISE CRUDE PIPELINE LLC,crude,811.00,0.0,0.0,0.0,0.006,87.714,0.047,0.039,0.129,67.612,39.238,0.0,31.017455,Enterprise Products Partners,24.0,10.0
474,26085,2012,"PLAINS MARKETING, L.P.",non-hvl,6.27,0.0,0.0,0.0,0.000,0.000,3.780,0.000,2.490,0.000,0.000,0.0,37.057416,Plains GP Holding,0.0,0.0


In [6]:
pipelines_2010_raw = pd.read_feather(pipelines_2010_raw_file)

## 8.1 Functions for analysis

In [7]:
from functools import partial

def find_info(OPERATOR_ID, info_col: str, title: str, df = pipelines_2010_raw, id_col = 'OPERATOR_ID', 
              year_col = 'REPORT_YEAR', fuzzy=False):
    from fuzzywuzzy import fuzz
    
    values = np.unique(df[df[id_col] == int(OPERATOR_ID)][info_col]).tolist()
    result = []
    for value in values:
        start_year = df[df[info_col] == value][year_col].min()
        end_year = df[df[info_col] == value][year_col].max()
        result = result + [{title: value, 'start_year': start_year, 'end_year': end_year}]
        
    if fuzzy and len(result) == 2 and fuzz.ratio(result[0][title].lower(), result[1][title].lower()) >= 95:
            result = [result[0]]
            
    return(result)

find_address = partial(find_info, info_col='PARTA4STREET', title='address')
find_names = partial(find_info, info_col='PARTA2NAMEOFCOMP', title='name', fuzzy=True)
find_names('4906')

[{'name': 'EXXONMOBIL PIPELINE CO', 'start_year': 2010, 'end_year': 2018}]

In [8]:
def find_total_miles_year(OPERATOR_ID, year, df = pipelines_2010_raw, year_col = 'REPORT_YEAR', 
                          id_col = 'OPERATOR_ID', miles_col='PARTBHCATOTAL'):
    observations = df.loc[(df[year_col] == year) & (df[id_col] == int(OPERATOR_ID))][miles_col]
    return observations.sum()

find_total_miles_year('4906', 2015)

2113.75

In [9]:
def find_total_miles(OPERATOR_ID, years, df = pipelines_2010_raw, year_col = 'REPORT_YEAR', miles_col='PARTBHCATOTAL'):
    result = []
    for year in years:
        result.append([year, find_total_miles_year(OPERATOR_ID, year)])
    return(result)

find_total_miles('4906', range(2013, 2017))

[[2013, 2944.4], [2014, 2343.06], [2015, 2113.75], [2016, 2590.17]]

## 8.2 Iterate over issues and resolve them one at a time

We use the dictionary "company groups" to capture any companies we have to combine to form one observation.

### Store results in

In [10]:
company_groups = pd.DataFrame()

def add_company_group(name: str, members: list, df = company_groups):
    new_group = pd.DataFrame({'members': members})
    new_group['name'] = name
    df = df.append(new_group, ignore_index=True)
    return df

In [11]:
m_as = pd.DataFrame()

def add_m_a(name: str, members: list, df, start_year: str = None, end_year: str = None):
    new_group = pd.DataFrame({'members': members})
    new_group['name'] = name
    
    if start_year:
        new_group['start_year'] = start_year
    if end_year:
        new_group['end_year'] = end_year
    
    df = df.append(new_group, sort=False, ignore_index=True)
    return df

In [12]:
spin_offs = []

### 8.2.1 Enterprise Products Operating

In [13]:
print(next(issues))


ENTERPRISE PRODUCTS OPERATING LLC (OPERATOR_ID 31618) has the same parent company as:

	DIXIE PIPELINE COMPANY LLC (OPERATOR_ID 3445)

	ENTERPRISE CRUDE PIPELINE LLC (OPERATOR_ID 30829)



### Gather infor_asation

In [14]:
find_names('31618')

[{'name': 'ENTERPRISE PRODUCTS OPERATING LLC',
  'start_year': 2010,
  'end_year': 2018}]

In [15]:
find_names('30829')

[{'name': 'ENTERPRISE CRUDE PIPELINE LLC',
  'start_year': 2010,
  'end_year': 2018}]

In [16]:
find_names('3445')

[{'name': 'DIXIE PIPELINE', 'start_year': 2010, 'end_year': 2010},
 {'name': 'DIXIE PIPELINE COMPANY LLC', 'start_year': 2011, 'end_year': 2018}]

### Decision

Enterprise products partners aqcuired a majority stake in Dixie Pipelines in 2005. Enterprise Crude Pipeline is a wholly owned subsidiary.

In [17]:
company_groups = add_company_group('Enterprise Products (Group)', ['31618', '30829', '3445'], company_groups)
company_groups

,members,name
0,31618,Enterprise Products (Group)
1,30829,Enterprise Products (Group)
2,3445,Enterprise Products (Group)


### 8.2.2 Oneok NGL

In [18]:
print(next(issues))


ONEOK NGL PIPELINE, LLC (OPERATOR_ID 32109) has changed its name:

	Was named ONEOK NGL PIPELINE LP from 2010 to 2011.

	Was named ONEOK NGL PIPELINE, LLC from 2012 to 2018.



No action necessary.

### 8.2.3 Magellan

In [19]:
print(next(issues))


MAGELLAN PIPELINE COMPANY, LP (OPERATOR_ID 22610) has the same parent company as:

	MAGELLAN CRUDE OIL PIPELINE COMPANY, L.P. (OPERATOR_ID 39504)

MAGELLAN PIPELINE COMPANY, LP (OPERATOR_ID 22610) may have a namesake or namesakes:

	MAGELLAN AMMONIA PIPELINE, L.P. (OPERATOR_ID 12105)

	Magellan Ammonia Pipeline, L.P. (OPERATOR_ID 12105)



In [20]:
find_names('22610')

[{'name': 'MAGELLAN PIPELINE COMPANY, LP',
  'start_year': 2011,
  'end_year': 2018}]

In [21]:
find_names('39504')

[{'name': 'MAGELLAN CRUDE OIL PIPELINE COMPANY, L.P.',
  'start_year': 2016,
  'end_year': 2018}]

Why does this business unit only exist from 2016?

In [22]:
find_names('12105')

[{'name': 'MAGELLAN AMMONIA PIPELINE, L.P.',
  'start_year': 2011,
  'end_year': 2018}]

In [23]:
find_total_miles('22610', range(2014, 2018))

[[2014, 4505.5], [2015, 4493.25], [2016, 4370.52], [2017, 4403.66]]

In [24]:
find_total_miles('12105', range(2014, 2018))

[[2014, 244.5], [2015, 245.0], [2016, 245.61], [2017, 244.74]]

In [25]:
find_total_miles('39504', range(2016, 2019))

[[2016, 223.68], [2017, 234.02], [2018, 234.07]]

A search on LexisNexis Dossiers yielded no insights on the subsidiaries being outside acquisitions that were merged in.

Magellan acquired some pipelines from BP in September 2010, which might explain the start of reporting in 2011. https://www.magellanlp.com/uploadedFiles/Investors/Financials/2011/annual%20report%202011.pdf.

### Decision

In [26]:
company_groups = add_company_group('Magellan (Group)', ['22610', '12105', '39504'], company_groups)
company_groups

,members,name
0,31618,Enterprise Products (Group)
1,30829,Enterprise Products (Group)
2,3445,Enterprise Products (Group)
3,22610,Magellan (Group)
4,12105,Magellan (Group)
5,39504,Magellan (Group)


### 8.2.4 Phillips 66

In [27]:
print(next(issues))


PHILLIPS 66 PIPELINE LLC (OPERATOR_ID 31684) has changed its name:

	Was named CONOCOPHILLIPS from 2010 to 2010.

	Was named PHILLIPS 66 PIPELINE LLC from 2011 to 2018.

PHILLIPS 66 PIPELINE LLC (OPERATOR_ID 31684) has the same parent company as:

	PHILLIPS 66 COMPANY - SWEENY REFINERY (OPERATOR_ID 15485)

PHILLIPS 66 PIPELINE LLC (OPERATOR_ID 31684) may have a namesake or namesakes:

	CONOCOPHILLIPS COMPANY (OPERATOR_ID 15485)

	PHILLIPS 66 COMPANY (OPERATOR_ID 15485)

	CONOCOPHILLIPS COMPANY (OPERATOR_ID 15485)



Downstream business was spun off in 2011 as Phillips 66.

In [28]:
find_names('15485')

[{'name': 'CONOCOPHILLIPS COMPANY', 'start_year': 2010, 'end_year': 2010},
 {'name': 'PHILLIPS 66 COMPANY', 'start_year': 2011, 'end_year': 2011},
 {'name': 'PHILLIPS 66 COMPANY - SWEENY REFINERY',
  'start_year': 2012,
  'end_year': 2018}]

In [29]:
find_names('31684')

[{'name': 'CONOCOPHILLIPS', 'start_year': 2010, 'end_year': 2010},
 {'name': 'PHILLIPS 66 PIPELINE LLC', 'start_year': 2011, 'end_year': 2018}]

In [30]:
find_total_miles('31684', range(2010, 2014))

[[2010, 4179.09],
 [2011, 3967.0699999999997],
 [2012, 4036.4300000000003],
 [2013, 4050.83]]

In [31]:
find_total_miles('15485', range(2010, 2014))

[[2010, 340.23699999999997],
 [2011, 340.23699999999997],
 [2012, 125.0],
 [2013, 95.96000000000001]]

In [32]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE '%%sweeny%%' LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2


Sweeny refinery is not an independent organizations. We will reconsolidate the two organizations.

### Decision

In [33]:
company_groups = add_company_group('Phillips 66 (Group)', ['15485', '31684'], company_groups)
company_groups

,members,name
0,31618,Enterprise Products (Group)
1,30829,Enterprise Products (Group)
2,3445,Enterprise Products (Group)
3,22610,Magellan (Group)
4,12105,Magellan (Group)
5,39504,Magellan (Group)
6,15485,Phillips 66 (Group)
7,31684,Phillips 66 (Group)


In [34]:
spin_offs.append({'name': 'Phillips 66 (Group)', 'year': '2011'})

### 8.2.5 Buckeye

In [35]:
print(next(issues))


BUCKEYE PARTNERS, LP (OPERATOR_ID 1845) has the same parent company as:

	BUCKEYE DEVELOPMENT & LOGISTICS, LLC (OPERATOR_ID 31371)



In [36]:
db.raw_sql("SELECT * FROM compa.names WHERE lower(conm) LIKE '%%buckeye%%' LIMIT 10")

,gvkey,conm,tic,cusip,cik,sic,naics,gsubind,gind,year1,year2
0,002451,BUCKEYE INTL INC,BEYE.,118169101,0000015070,3320,None,None,None,1965.0,1979.0
1,013526,BUCKEYE PARTNERS LP,BPL,118230101,0000805022,4610,486910,10102040,101020,1986.0,2018.0
2,024417,BUCKEYE COMMUNICATIONS INC,3BKCIE,118154202,0000822822,5130,4223,25501010,255010,1989.0,1994.0
3,061596,BUCKEYE TECHNOLOGIES INC,BKI.1,118255108,0000899597,2611,322110,15105020,151050,1993.0,2013.0
4,174316,BUCKEYE GP HOLDINGS LP,BGH.2,118167105,0001359055,5172,424720,10102040,101020,2001.0,2009.0


### Decision

LexisNexis does not indicate that Buckeye Development & Logistics is an outside acquisition by Buckeye Partners. We consolidate the two organizations into one.

In [37]:
company_groups = add_company_group('Buckeye (Group)', ['1845', '31371'], company_groups)
company_groups.tail()

,members,name
5,39504,Magellan (Group)
6,15485,Phillips 66 (Group)
7,31684,Phillips 66 (Group)
8,1845,Buckeye (Group)
9,31371,Buckeye (Group)


### 8.2.5 Sunoco

In [38]:
print(next(issues))


SUNOCO PIPELINE L.P. (OPERATOR_ID 18718) has the same parent company as:

	MID - VALLEY PIPELINE CO (OPERATOR_ID 12470)

	ENERGY TRANSFER COMPANY (OPERATOR_ID 32099)

	INLAND CORPORATION (OPERATOR_ID 32683)

	DAPL-ETCO OPERATIONS MANAGEMENT, LLC (OPERATOR_ID 39205)

	PERMIAN EXPRESS PARTNERS LLC (OPERATOR_ID 39596)



### Names

In [39]:
find_names('18718')

[{'name': 'SUNOCO PIPELINE L.P.', 'start_year': 2010, 'end_year': 2018}]

In [40]:
find_names('12470')

[{'name': 'MID - VALLEY PIPELINE CO', 'start_year': 2010, 'end_year': 2018}]

In [41]:
find_total_miles('12470', range(2010, 2019))

[[2010, 542.0],
 [2011, 542.0],
 [2012, 542.0],
 [2013, 633.0],
 [2014, 656.0],
 [2015, 662.0],
 [2016, 632.0],
 [2017, 650.0],
 [2018, 650.47]]

Sunoco has a controlling stake in Mid-Valley Pipeline Company. Some stakes were acquired in 2010, but Sunoco had a controlling stake even before that transaction.

In [42]:
find_names('32099')

[{'name': 'ENERGY TRANSFER COMPANY', 'start_year': 2011, 'end_year': 2018}]

In [43]:
find_total_miles('32099', range(2010, 2019))

[[2010, 0.0],
 [2011, 331.8],
 [2012, 463.2],
 [2013, 463.43],
 [2014, 460.89],
 [2015, 468.79],
 [2016, 744.4200000000001],
 [2017, 1310.0],
 [2018, 1295.11]]

In [44]:
find_names('39205')

[{'name': 'DAPL-ETCO OPERATIONS MANAGEMENT, LLC',
  'start_year': 2017,
  'end_year': 2018}]

In [45]:
find_total_miles('39205', range(2010, 2019))

[[2010, 0.0],
 [2011, 0.0],
 [2012, 0.0],
 [2013, 0.0],
 [2014, 0.0],
 [2015, 0.0],
 [2016, 0.0],
 [2017, 795.9],
 [2018, 950.64]]

Dacota Access PipeLine - construction started in 2016.

In [46]:
find_names('39596')

[{'name': 'PERMIAN EXPRESS PARTNERS LLC',
  'start_year': 2017,
  'end_year': 2018}]

In [47]:
find_total_miles('39596', range(2010, 2019))

[[2010, 0.0],
 [2011, 0.0],
 [2012, 0.0],
 [2013, 0.0],
 [2014, 0.0],
 [2015, 0.0],
 [2016, 0.0],
 [2017, 946.0],
 [2018, 946.62]]

Permian express oil pipeline - wholly owned subsidiary of Sunoco, construction started in 2013 (https://www.sourcewatch.org/index.php/Permian_Express_Oil_Pipeline).

In [48]:
find_names('32683')

[{'name': 'INLAND CORPORATION', 'start_year': 2012, 'end_year': 2018}]

In [49]:
find_total_miles('32683', range(2010, 2015))

[[2010, 0.0], [2011, 0.0], [2012, 221.69], [2013, 264.0], [2014, 264.0]]

Sunoco Logistics acquired Inland Corp from BP in 2011 (Mergerstat M&A database Deal Number 586584). BP did probably not report this part of their pipeline system as a separate entity, hence the organization showing up as new in the FERC database.

### Decision

We list Sunoco (OPERATOR_ID 18718), Mid-Valley Pipeline (OPERATOR_ID 12470), Inland Corporation (OPERATOR_ID 32683), and Permian (OPERATOR_ID 39596) together as one company until 2017. Similarly, we list Energy Transfer (OPERATOR_ID 32099) and DAPL (OPERATOR_ID 39205) together until 2017. From 2017, we merge the two under the name of Sunoco.

In [50]:
# Sunoco
company_groups = add_company_group('Sunoco (Group)', ['18718', '12470', '39596', '32683'], company_groups)

# Tesoro
company_groups = add_company_group('Energy Transfer (Group)', ['32099', '39205'], company_groups)
company_groups.tail()

,members,name
11,12470,Sunoco (Group)
12,39596,Sunoco (Group)
13,32683,Sunoco (Group)
14,32099,Energy Transfer (Group)
15,39205,Energy Transfer (Group)


In [51]:
m_as = add_m_a('Sunoco (Group)', ['Sunoco (Group)', 'Energy Transfer (Group)'], m_as, start_year = '2017')
m_as

,members,name,start_year
0,Sunoco (Group),Sunoco (Group),2017
1,Energy Transfer (Group),Sunoco (Group),2017


Marathon acquired Andeavor in 2018.

### 8.2.6 Flint Hills

In [52]:
print(next(issues))


FLINT HILLS RESOURCES, LC (OPERATOR_ID 22855) has changed its name:

	Was named FLINT HILLS RESOURCES, LC from 2017 to 2018.

	Was named KOCH PIPELINE COMPANY, L.P. from 2010 to 2016.



In [53]:
find_total_miles('22855', range(2015, 2019))

[[2015, 1885.19], [2016, 1674.88], [2017, 1711.48], [2018, 1750.7199999999998]]

The restructuring or sell-off does not seem to affect the organizations pipeline assets. We will ignore the rebranding efforts.

### 8.2.7 NuStar

In [54]:
print(next(issues))


NUSTAR PIPELINE OPERATING PARTNERSHIP L.P. (OPERATOR_ID 10012) has the same parent company as:

	NUSTAR LOGISTICS, L.P. (OPERATOR_ID 31454)



In [55]:
find_names('10012')

[{'name': 'NUSTAR PIPELINE OPERATING PARTNERSHIP L.P.',
  'start_year': 2010,
  'end_year': 2018}]

In [56]:
find_names('31454')

[{'name': 'NUSTAR LOGISTICS, L.P.', 'start_year': 2010, 'end_year': 2018}]

LexisNexis Doessier does not list any M&As for the two subsidiaries. They just seem to be two separate subsidiaries, that we can group together.

In [57]:
company_groups = add_company_group('NuStar', ['10012', '31454'], company_groups)
company_groups.tail()

,members,name
13,32683,Sunoco (Group)
14,32099,Energy Transfer (Group)
15,39205,Energy Transfer (Group)
16,10012,NuStar
17,31454,NuStar


### 8.2.8 Kinder Morgan

In [58]:
print(next(issues))


PLANTATION PIPE LINE CO (OPERATOR_ID 15674) has the same parent company as:

	SFPP, LP (OPERATOR_ID 18092)

	CALNEV PIPELINE CO (OPERATOR_ID 26125)

	KINDER MORGAN COCHIN LLC (OPERATOR_ID 32258)



In [59]:
find_names('15674')

[{'name': 'PLANTATION PIPE LINE CO', 'start_year': 2010, 'end_year': 2018}]

Kinder Morgan owns a majority stake since 1999 (LexnisNexis Deal Number 877657020).

In [60]:
find_names('18092')

[{'name': 'SFPP, LP', 'start_year': 2010, 'end_year': 2018}]

LexisNexis indicates no M&As. Probably represents greenfield investment.

In [61]:
find_names('26125')

[{'name': 'CALNEV PIPELINE CO', 'start_year': 2010, 'end_year': 2018}]

LexisNexis indicates no M&As. Probably represents greenfield investment.

In [62]:
find_names('32258')

[{'name': 'KINDER MORGAN COCHIN LLC', 'start_year': 2010, 'end_year': 2018}]

About to be sold of to Pembina, but LexisNexis indicates that it originally represented a greenfield investment by Kinder Morgan.

### Decision

In [63]:
company_groups = add_company_group('Kinder Morgan (Group)', ['15674', '18092', '26125', '32258'], company_groups)
company_groups.tail()

,members,name
17,31454,NuStar
18,15674,Kinder Morgan (Group)
19,18092,Kinder Morgan (Group)
20,26125,Kinder Morgan (Group)
21,32258,Kinder Morgan (Group)


### 8.2.9 Enbridge

In [64]:
print(next(issues))


ENBRIDGE ENERGY, LIMITED PARTNERSHIP (OPERATOR_ID 11169) has the same parent company as:

	EXPRESS HOLDINGS (USA), LLC (OPERATOR_ID 31720)

	ENBRIDGE PIPELINES (OZARK) L.L.C. (OPERATOR_ID 31947)

	CCPS TRANSPORTATION, LLC (OPERATOR_ID 32080)

	ENBRIDGE PIPELINES (SOUTHERN LIGHTS) L.L.C. (OPERATOR_ID 32502)



In [65]:
find_names('11169')

[{'name': 'ENBRIDGE ENERGY, LIMITED PARTNERSHIP',
  'start_year': 2010,
  'end_year': 2018}]

In [66]:
find_names('31720')

[{'name': 'EXPRESS HOLDINGS (USA), LLC', 'start_year': 2013, 'end_year': 2018},
 {'name': 'KINDER MORGAN PIPELINES (USA) INC',
  'start_year': 2010,
  'end_year': 2012}]

In [67]:
find_total_miles('31720', range(2010, 2015))

[[2010, 196.0], [2011, 196.0], [2012, 196.0], [2013, 219.3], [2014, 212.4]]

In [68]:
find_names('32080')

[{'name': 'CCPS TRANSPORTATION, LLC', 'start_year': 2010, 'end_year': 2018}]

LexisNexis indicates no M&A activities.

In [69]:
find_names('32502')

[{'name': 'ENBRIDGE PIPELINES (SOUTHERN LIGHTS) L.L.C.',
  'start_year': 2010,
  'end_year': 2018}]

LexisNexis indicates no M&A activities.

In [70]:
find_names('31947')

[{'name': 'ENBRIDGE PIPELINES (OZARK) L.L.C.',
  'start_year': 2010,
  'end_year': 2018}]

LexisNexis indicates no M&A activities.

### Express Holdings

In [71]:
find_total_miles('31720', range(2010, 2015))

[[2010, 196.0], [2011, 196.0], [2012, 196.0], [2013, 219.3], [2014, 212.4]]

The Express pipeline system was sold by Kinder Morgan to Spectra Energy ~2012 (LexisNexis Dossier Deal Number 2476224040). Enbridge merged with Spectra Energy in 2017 (LexisNexis Dossier Deal Number 885812).

### Decision

We group the CCPS, Enbridge Pipelines Ozark, and Enbridge Pipelines (Southern Lights) with Enbridge. Express Holdings is modeled as part of Kinder Morgan until 2012, then as part of Spectra Energy (which has no other pipeline business in the datset) until 2016, and finally as part of Enbridge in 2018.

In [72]:
company_groups = add_company_group('Enbridge (Group)', ['11169', '32080', '32502', '31947'], company_groups)
company_groups.tail()

,members,name
21,32258,Kinder Morgan (Group)
22,11169,Enbridge (Group)
23,32080,Enbridge (Group)
24,32502,Enbridge (Group)
25,31947,Enbridge (Group)


In [73]:
# new_9 - Kinder Morgan
m_as = add_m_a('Kinder Morgan (Group)', ['Kinder Morgan (Group)', '31720'], m_as, end_year = '2013')

m_as = add_m_a('Enbridge (Group)', ['Enbridge (Group)', '31720'], m_as, start_year = '2017')
m_as

,members,name,start_year,end_year
0,Sunoco (Group),Sunoco (Group),2017,NaN
1,Energy Transfer (Group),Sunoco (Group),2017,NaN
2,Kinder Morgan (Group),Kinder Morgan (Group),NaN,2013
3,31720,Kinder Morgan (Group),NaN,2013
4,Enbridge (Group),Enbridge (Group),2017,NaN
5,31720,Enbridge (Group),2017,NaN


### 9.2.10 Marathon

In [74]:
print(next(issues))


MARATHON PIPE LINE LLC (OPERATOR_ID 32147) has the same parent company as:

	WOLVERINE PIPELINE CO (OPERATOR_ID 22830)

	TESORO LOGISTICS OPERATIONS, LLC (OPERATOR_ID 38933)

	TESORO LOGISTICS NORTHWEST PIPELINE LLC (OPERATOR_ID 39029)



### Names

In [75]:
find_names('32147')

[{'name': 'MARATHON PIPE LINE LLC', 'start_year': 2010, 'end_year': 2018}]

In [76]:
find_names('22830')

[{'name': 'WOLVERINE PIPELINE CO', 'start_year': 2010, 'end_year': 2018}]

LexisNexis Dossier indicates that Sunoco holds a minority interest in Wolverine Pipeline (or Pipe Line) Co. Other than that, the company is continually owned by Marathon.

In [77]:
find_names('38933')

[{'name': 'TESORO LOGISTICS OPERATIONS LLC - SOUTHERN CALIFORNIA',
  'start_year': 2012,
  'end_year': 2017},
 {'name': 'TESORO LOGISTICS OPERATIONS, LLC',
  'start_year': 2018,
  'end_year': 2018}]

LexisNexis Dossiers indicates no M&A activities.

In [78]:
find_names('39029')

[{'name': 'TESORO LOGISTICS NORTHWEST PIPELINE LLC',
  'start_year': 2013,
  'end_year': 2017}]

### Decision

LexisNexis indicates that Tesoro was renamed to Andeavor, and acquired by Marathon in 2018 (LexisNexis Dossier Deal Number 3052392). We will group Wolverine with Marathon, and also combine the two Tesoros into one observation. For 2018, we will subsume Tesoro under Marathon.

In [79]:
# Marathon and Wolverine
company_groups = add_company_group('Marathon (Group)', ['32147', '22830'], company_groups)

# Tesoro
company_groups = add_company_group('Tesoro (Group)', ['38933', '39029'], company_groups)
company_groups.tail()

,members,name
25,31947,Enbridge (Group)
26,32147,Marathon (Group)
27,22830,Marathon (Group)
28,38933,Tesoro (Group)
29,39029,Tesoro (Group)


In [80]:
m_as = add_m_a('Marathon (Group)', ['Marathon (Group)', 'Tesoro (Group)'], m_as, start_year = '2018')
m_as

,members,name,start_year,end_year
0,Sunoco (Group),Sunoco (Group),2017,NaN
1,Energy Transfer (Group),Sunoco (Group),2017,NaN
2,Kinder Morgan (Group),Kinder Morgan (Group),NaN,2013
3,31720,Kinder Morgan (Group),NaN,2013
4,Enbridge (Group),Enbridge (Group),2017,NaN
5,31720,Enbridge (Group),2017,NaN
6,Marathon (Group),Marathon (Group),2018,NaN
7,Tesoro (Group),Marathon (Group),2018,NaN


### 8.2.11 ExxonMobil

In [81]:
print(next(issues))


EXXONMOBIL PIPELINE CO (OPERATOR_ID 4906) has the same parent company as:

	EXXONMOBIL PIPELINE CO (OPERATOR_ID 12628)

	EXXONMOBIL OIL CORP - WEST COAST (OPERATOR_ID 26134)



In [82]:
find_names('4906')

[{'name': 'EXXONMOBIL PIPELINE CO', 'start_year': 2010, 'end_year': 2018}]

In [83]:
find_names('12628')

[{'name': 'EXXONMOBIL PIPELINE CO', 'start_year': 2010, 'end_year': 2018},
 {'name': 'MOBIL  PIPE  LINE COMPANY', 'start_year': 2012, 'end_year': 2018},
 {'name': 'MOBIL PIPELINE CO', 'start_year': 2010, 'end_year': 2010}]

Company name inconsistent between different documents.

In [84]:
find_names('26134')

[{'name': 'EXXONMOBIL OIL CORP - WEST COAST',
  'start_year': 2010,
  'end_year': 2018}]

All are wholly owned subsidiaries of exxonmobil. LexisNexis indicates no M&A activities. We combine them into one observation.

In [85]:
company_groups = add_company_group('ExxonMobil (Group)', ['4906', '12628', '26134'], company_groups)
company_groups.tail()

,members,name
28,38933,Tesoro (Group)
29,39029,Tesoro (Group)
30,4906,ExxonMobil (Group)
31,12628,ExxonMobil (Group)
32,26134,ExxonMobil (Group)


### 8.2.12 ExxonMobil

In [86]:
print(next(issues))


EXXONMOBIL PIPELINE CO (OPERATOR_ID 12628) has changed its name:

	Was named EXXONMOBIL PIPELINE CO from 2010 to 2018.

	Was named MOBIL  PIPE  LINE COMPANY from 2012 to 2018.

	Was named MOBIL PIPELINE CO from 2010 to 2010.



Already addressed above.

### 8.2.13 Tesoro

In [87]:
print(next(issues))


TESORO LOGISTICS OPERATIONS, LLC (OPERATOR_ID 38933) has changed its name:

	Was named TESORO LOGISTICS OPERATIONS LLC - SOUTHERN CALIFORNIA from 2012 to 2017.

	Was named TESORO LOGISTICS OPERATIONS, LLC from 2018 to 2018.



Already addressed above.

### 8.2.14 TC OIL

In [88]:
print(next(issues))


TC OIL PIPELINE OPERATIONS INC (OPERATOR_ID 32334) may have a namesake or namesakes:

	DAPL-ETCO OPERATIONS MANAGEMENT, LLC (OPERATOR_ID 39205)



False positive

### 8.2.15 Plains Pipeline

In [89]:
print(next(issues))


ROCKY MOUNTAIN PIPELINE SYSTEM, LLC (OPERATOR_ID 31666) has the same parent company as:

	PLAINS PIPELINE, L.P. (OPERATOR_ID 300)

	PLAINS MARKETING, L.P. (OPERATOR_ID 26085)



In [90]:
find_names('26085')

[{'name': 'PLAINS MARKETING, L.P.', 'start_year': 2010, 'end_year': 2018}]

In [91]:
find_names('300')

[{'name': 'PLAINS PIPELINE, L.P.', 'start_year': 2010, 'end_year': 2018}]

In [92]:
find_names('31666')

[{'name': 'ROCKY MOUNTAIN PIPELINE SYSTEM, LLC',
  'start_year': 2010,
  'end_year': 2010}]

In [93]:
find_total_miles('31666', range(2010, 2013))

[[2010, 723.55], [2011, 0.0], [2012, 0.0]]

In [94]:
find_total_miles('300', range(2010, 2013))

[[2010, 1323.72], [2011, 2314.94], [2012, 2350.39]]

In [95]:
find_total_miles('26085', range(2010, 2013))

[[2010, 336.27], [2011, 283.46000000000004], [2012, 261.39]]

LexisNexis Doassiers indicates no M&As for the subsidiaries, Plains Marketing and Rocky Mountain Pipeline System.

### Decision

Obviously Plains Pipeline absorbed its wholly owned subsidiary, Rocky Mountain Pipeline System. We list the three suborganizations as one organization.

In [96]:
company_groups = add_company_group('Plains Pipeline (Group)', ['300', '31666', '26085'], company_groups)
company_groups.tail()

,members,name
31,12628,ExxonMobil (Group)
32,26134,ExxonMobil (Group)
33,300,Plains Pipeline (Group)
34,31666,Plains Pipeline (Group)
35,26085,Plains Pipeline (Group)


### 8.2.16 Dixie Pipeline

In [97]:
print(next(issues))


DIXIE PIPELINE COMPANY LLC (OPERATOR_ID 3445) has changed its name:

	Was named DIXIE PIPELINE from 2010 to 2010.

	Was named DIXIE PIPELINE COMPANY LLC from 2011 to 2018.



No action necessary.

### 8.2.17 Kinder Morgan

In [98]:
print(next(issues))


KINDER MORGAN COCHIN LLC (OPERATOR_ID 32258) may have a namesake or namesakes:

	KINDER MORGAN PIPELINES (USA) INC (OPERATOR_ID 31720)



Already addressed above.

### 8.2.18 BP

In [99]:
print(next(issues))


BP PIPELINE (NORTH AMERICA) INC. (OPERATOR_ID 31189) has the same parent company as:

	BP OIL PIPELINE CO (OPERATOR_ID 18386)

	OLYMPIC PIPE LINE COMPANY (OPERATOR_ID 30781)



In [100]:
find_names('31189')

[{'name': 'BP PIPELINE (NORTH AMERICA) INC.',
  'start_year': 2010,
  'end_year': 2018}]

In [101]:
find_names('18386')

[{'name': 'BP OIL PIPELINE CO', 'start_year': 2010, 'end_year': 2018}]

LexisNexis indicates no M&As that would concern us, only the sale of some assets in 2005.

In [102]:
find_names('30781')

[{'name': 'OLYMPIC PIPE LINE COMPANY', 'start_year': 2010, 'end_year': 2018}]

We have previously wrongly assumed Olympic is a subsidiary of BP. In 2006, Enbridge acquired a majority stake (LexisNexis Doassier Deal Number 358159).

In [103]:
company_groups = add_company_group('BP (Group)', ['31189', '18386'], company_groups)
company_groups.tail()

,members,name
33,300,Plains Pipeline (Group)
34,31666,Plains Pipeline (Group)
35,26085,Plains Pipeline (Group)
36,31189,BP (Group)
37,18386,BP (Group)


In [104]:
company_groups = add_company_group('Enbridge (Group)', ['30781'], company_groups)
company_groups.tail()

,members,name
34,31666,Plains Pipeline (Group)
35,26085,Plains Pipeline (Group)
36,31189,BP (Group)
37,18386,BP (Group)
38,30781,Enbridge (Group)


### 8.2.19 Tesoro

In [105]:
print(next(issues))


TESORO LOGISTICS NORTHWEST PIPELINE LLC (OPERATOR_ID 39029) may have a namesake or namesakes:

	TESORO LOGISTICS OPERATIONS LLC - SOUTHERN CALIFORNIA (OPERATOR_ID 38933)



Already addressed above (Marathon).

### 8.2.20 Express Holdings

In [106]:
print(next(issues))


EXPRESS HOLDINGS (USA), LLC (OPERATOR_ID 31720) has changed its name:

	Was named EXPRESS HOLDINGS (USA), LLC from 2013 to 2018.

	Was named KINDER MORGAN PIPELINES (USA) INC from 2010 to 2012.

EXPRESS HOLDINGS (USA), LLC (OPERATOR_ID 31720) may have a namesake or namesakes:

	TALLGRASS PONY EXPRESS PIPELINE, LLC (OPERATOR_ID 39043)

	PERMIAN EXPRESS PARTNERS LLC (OPERATOR_ID 39596)

	KINDER MORGAN COCHIN LLC (OPERATOR_ID 32258)



Kinder Morgan already adressed above (8.2.10). No new relevant namesakes.

### 8.2.21

In [107]:
print(next(issues))

StopIteration: 

## 8.3 Overview

In [108]:
company_groups

,members,name
0,31618,Enterprise Products (Group)
1,30829,Enterprise Products (Group)
2,3445,Enterprise Products (Group)
3,22610,Magellan (Group)
4,12105,Magellan (Group)
5,39504,Magellan (Group)
6,15485,Phillips 66 (Group)
7,31684,Phillips 66 (Group)
8,1845,Buckeye (Group)
9,31371,Buckeye (Group)


In [109]:
m_as

,members,name,start_year,end_year
0,Sunoco (Group),Sunoco (Group),2017,NaN
1,Energy Transfer (Group),Sunoco (Group),2017,NaN
2,Kinder Morgan (Group),Kinder Morgan (Group),NaN,2013
3,31720,Kinder Morgan (Group),NaN,2013
4,Enbridge (Group),Enbridge (Group),2017,NaN
5,31720,Enbridge (Group),2017,NaN
6,Marathon (Group),Marathon (Group),2018,NaN
7,Tesoro (Group),Marathon (Group),2018,NaN


In [110]:
spin_offs

[{'name': 'Phillips 66 (Group)', 'year': '2011'}]

In [111]:
company_groups.to_feather(f'../preprocessed_data/company_groups_{today}.feather')
m_as.to_feather(f'../preprocessed_data/m_as_{today}.feather')